<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=4>Módulo 8</font></h2>



<h1><font color="#00586D" size=5>Sistema de recomendación de libros</font></h1>

<br><br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Luis de la Ossa</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Introducción](#section1)
   * [Lectura de las revisiones](#section11)
   * [Etiquetas](#section12)
   * [Preparación de los datos](#section13)
* [2. Limpieza del texto](#section2)
* [3. Búsqueda por similaridad de la representación tf-idf](#section3)
* [4. Recomendación basada en contenido](#section4)
   * [Selección de libros preferidos por el usuario](#section41)
   * [Búsqueda de libros similares](#section42)
   * [Priorización de resultados y recomendación](#section43)
* [5. Sistema híbrido de recomendación](#section5)  
* [6. LDA y búsqueda por similaridad](#section6)
<br>

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

<a id="section1"></a>
## <font color="#00586D"> 1. Introducción</font>
<br>

Los sistemas de recomendación basados en filtrado colaborativo utilizan de manera exclusiva los perfiles de votación de usuarios/items, y no consideran el contenido. Como se vió en clase,  permiten obtener resultados que, si bien cualitativamente son aceptables, pueden llegar a desconcertar al no corresponderse con lo esperado. 

En este proyecto se diseñará un pequeño sistema de recomendación de libros basado en contenido. Para ello, será necesario hacer uso de algunos de los conceptos relaccionados con aprendizaje automático sobre información textual (_Text Mining_) que han sido tratados también a lo largo del módulo. 

Como punto de partida, se partirá del conjunto de datos [goodbooks-10k](https://www.kaggle.com/zygmunt/goodbooks-10k) disponible en [kaggle](https://www.kaggle.com). Éste contiene información relativa a 10000 libros obtenida de la red social  [goodreads](http://goodreads.com), que actualmente es el sitio de referencia en la red para aficcionados a la lectura. Además de títulos y autores, el conjunto de datos incluye votos y etiquetas aportadas por más de 53000 usuarios. 

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Se han hecho algunas modificaciones con respecto a la base original para que sea menos tedioso manejar los distintos índices e identificadores. 
</div>

In [ ]:
import pandas as pd
import numpy as np

df_goodreads = pd.read_csv('data/books.csv', sep="\t", index_col=0)
df_goodreads.head(2)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section11"></a> 
###  <font color="#00586D"> Lectura de las revisiones </font>
<br>

Los datos de la tabla apenas contienen información relativa al contenido de cada libro. Sin embargo, es posible acceder a los resúmenes almacenados en la propia web [GoodReads](www.goodreads.com) que, en cierto grado, aportan esta información. En el [anexo](#Anexo) se adjunta la función utilizada para obtenerlos. 

El resumen de cada libro ha sido almacenado en un archivo de texto denominado `./data/overviews/bookid.txt`, donde _bookid_ corresponde al identificador del libro (columna `book_id` de `df_goodreads`). 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Implementar una función, denominada `get_overview(gr_book_id)`, que reciba el identificador de un libro, lea el archivo de texto que contiene el resumen correspondiente y lo devuelva en un _String_ (o devuelva `None` si este resumen no existe). 

In [ ]:
def get_overview(gr_book_id):
    try:
        #
        # COMPLETAR 
        #
    except:
        return None
    
get_overview(320) # Cien años de soledad (Gabriel García Márquez)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true"></i></font> Crear una columna en `df_goodreads`, denominada `overview`, que contenga la revisión del libro correspondiente. Rellenar los valores vacíos de esa columna con un _String_ de longitud cero ("").

In [ ]:
df_goodreads['overview'] = # COMPLETAR 

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section12"></a> 
###  <font color="#00586D">Etiquetas </font>
<br>


En el conjunto de datos se proporcionan etiquetas relativas a cada libro que han sido aportadas por los usuarios. Esta información está incluída en dos archivos. El primero de ellos, `tags.csv`, contiene el identificador de cada etiqueta y el código correspondiente. El segundo, `book_tags.csv` contiene las etiquetas relativas a cada libro. Se almacenarán, respectivamente, en los _DataFrame_ `df_tags` y `df_book_tags`.

In [ ]:
df_tags = pd.read_csv('./data/tags.csv')
df_book_tags = pd.read_csv('./data/book_tags.csv')

Etiquetas

In [ ]:
df_tags.iloc[2000:2002]

Etiquetas por libro

In [ ]:
df_book_tags.head(3)

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Incorporar el campo `tag_name` de `df_tags` a `df_book_tags`, sustituyendo el campo `tag_id` por la etiqueta (_String_) correspondiente. Para ello, fundir los dos _DataFrame_ y, posteriormente, eliminar las columnas `count` y `tag_id`.

In [ ]:
df_book_tags = # COMPLETAR       

df_book_tags.tail()

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Inspeccionar las etiquetas que aparecen en `df_book_tags` y el número de veces que aparece cada una. ¿Deberían eliminarse algunas?

In [ ]:
#
# COMPLETAR
#

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Una de las cosas que se observan es que hay un alto número de etiquetas que aparecen una vez, y que son irrelevantes, por lo que es mejor ignorarlas. 


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Eliminar todas las etiquetas que aparezcan menos de 20 veces. 

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i> Este ejercicio se puede hacer de varias formas. Una de ellas, consiste en agrupar por `tag_name`, mediante `groupby`, y filtrar con `filter` los grupos con tamaño >=20.
</div>

In [ ]:
df_book_tags = # COMPLETAR 

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Otro de los problemas que se aprecian es que algunas etiquetas son genéricas, y no corresponden a libros concretos. Por ejemplo palabras como `read-readings` o `favourites`. 

In [ ]:
df_book_tags['tag_name'].value_counts().nlargest(25)

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Eliminar todas las etiquetas que contengan estos términos (los términos de la lista `target_tags`).  

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
En este caso, es recomendable no utilizar las etiquetas completas para eliminar así sus variantes también. Por otra parte, `Series.str.contains` acepta expresiones regulares.
</div>

In [ ]:
target_tags = ['read', 'favo', 'own', 'top', 'book', 'librar', 'kindle', 'list']

df_book_tags = # COMPLETAR 
df_book_tags

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Por último,  se creará un `DataFrame` denominado `df_book_tag_text` en el que, para cada libro (indicado por su código `goodreads_book_id`), se añadirá una columna con _un solo campo de texto_, resultado de unir las etiquetas correspondientes.  

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Agrupar las entradas `DataFrame` en `df_book_tags` en función del campo `goodreads_book_id` y unir todas las etiquetas de cada grupo mediante `join`. Almacenar el resultado en un `DataFrame` denominado `df_book_tag_text`.

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Para pasos posteriores es necesario convertir el resultado resultante, una `Serie`, en un `DataFrame`, mediante `Series.to_frame()`. 
</div>

In [ ]:
df_book_tag_text = # COMPLETAR 

df_book_tag_text.head()

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Renombrar la columna `tag_name` de `df_book_tags` a `text_tags`. Rellenar los valores perdidos con un String vacío ("").

In [ ]:
df_book_tag_text = # COMPLETAR 

df_book_tag_text.head()

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Incorporar la información del texto de las etiquetas, es decir, la columna `text_tags`, al `DataFrame` principal `df_goodreads` (utilizar _merge_).

In [ ]:
df_goodreads = # COMPLETAR 

df_goodreads.head(2)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section2"></a> 
## <font color="#00586D"> 2. Limpieza y preparación del texto </font>
<br>

En este proyecto, el contenido (texto) asociado a cada libro se representa mediante un modelo de bolsa de palabras. Como se ha explicado a lo largo del módulo, cuando se trabaja con este tipo de representación es recomendable limpiar el texto y eliminar información irrelevante. Para ello se utilizará la librería `spacy`.  En primer lugar, se utilizará el modelo de lenguaje `en_core_web_sm`. Aunque no es el modelo con mejor rendimiento, es suficiente para este contexto, y es más eficiente que `en_core_web_trf`(basado en *transformers*, y que es el que mejor funciona).

In [ ]:
#! python -m spacy download es_dep_news_trf
#! python -m spacy download en_core_web_sm # Solo se utilizará este
#! python -m spacy download en_core_web_trf 

In [ ]:
import spacy 

# Crea un objeto con el pipeline
nlp = spacy.load("en_core_web_sm")
nlp.pipe_names

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Crear una función denominada `clean` que acepte un texto, lo convierta en un documento (`spacy`) y devuelva un *String* compuesto por los lemas correspondientes a cada token, en minúscula, y descartando los tokens que no sean alfanuméricos, o los que correspondan a *stopwords*.

In [ ]:
def clean(overview):
    #
    # COMPLETAR   
    #
    
overview = df_goodreads.iloc[0]['overview']
clean(overview)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font>Aplicar la función `clean` sobre cada texto almacenado en la columna `df_goodreads['overview']`para preprocesarlo, almacenando el resultado en la propia columna.


<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Este ejercicio tarda unos cinco minutos en ejecutarse. Se ha planteado así (sin utilizar `nlp.pipe`) porque es más sencillo, y en este caso tarda prácticamente lo mismo.
</div>

In [ ]:
df_goodreads['overview'] = # COMPLETAR 

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section13"></a> 
###  <font color="#00586D"> Preparación de datos </font>
<br>

Una vez incorporada la información relativa a resúmenes y etiquetas, y puesto que el texto se tratará simplemente como una bolsa de palabras, esta se fundirá, constituyendo el contenido de cada libro. 


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i></font> 
Fundir las columnas `overview` y `text_tags` de `df_goodreads` en otra columna llamada`text`. Eliminar las columnas en las que `text` tenga longitud 0.

In [ ]:
#
# COMPLETAR
#

df_goodreads.shape

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

In [ ]:
data_books = df_goodreads['text']

####  <font color="#00586D"> Selección de cinco libros para pruebas </font>


A continuación, se eligen cinco libros, que serán almacenados en el `DataFrame` `df_books_test`

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Como no vamos a hacer predicción, no pasa nada porque los libros estén en dos sitios. 
</div>

In [ ]:
test_pos = np.array([2,4,7,9,12]) # Sugerencia. Podéis cambiarlos. 

df_books_test = df_goodreads.iloc[test_pos].copy()
df_books_test

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section3"></a> 
## <font color="#00586D"> 3. Búsqueda por similaridad de la representación tf-idf </font>
<br>

La información _tf-idf_ es muy útil de cara a clasificar y comparar documentos. Cada texto se representa  mediante un vector de valores _tf-idf_, y la búsqueda de documentos se apoya en una medida de similaridad, la _similaridad coseno_, para esta representación.

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i></font> Crear un objeto `TfidfVectorizer` de `sklearn`, denominado `tfidf_vect`, que represente un máximo de 20000 términos. Obtener la matriz _tf-idf_ de los datos almacenados en `data_books` y almacenarla en una variable denominada `data_books_tfidf`. Extraer los términos considerados en `tfidf_vect`, y almacenarlos en la variable `terms`.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#
# COMPLETAR
#

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

#### <font color="#00586D"> Búsqueda del libro más similar </font>

Se tomará un libro incluído en el conjunto de datos `df_books_test`. Por ejemplo, el primero.

In [ ]:
print(df_books_test.iloc[0])
text_query = df_books_test.iloc[0].text
text_query

Para poder buscar otro libro por similaridad, es necesario transformar el texto de consulta a la representación _tf-idf_. Previamente, ha de ser preprocesado (limpiado con la función `clean`).

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Transformar el documento `text_query` a formato _tf-idf_ y almacenar el resultado en `text_query_tfidf`.

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Esta sería la manera de proceder con libros nuevos. No obstante, en este caso concreto, ___como el libro ya estaba en el corpus y se ha obtenido su representación___, se podría obtener el vector directamente desde `data_books_tfidf`.

</div>

In [ ]:
text_query_tfidf = # COMPLETAR

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Mostrar los 10 términos más relevantes (con mayor valor _tf-idf_) para el documento `text_query`.

In [ ]:
#
# COMPLETAR
#

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Calcular las similaridades (similaridad coseno) del documento `text_query_tfidf` con el resto de documentos en la matriz `data_books_tfidf`, y almacenarlas en un vector denominado `similarities_tfidf`.

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
La función `cosine_similarity` toma como argumentos dos matrices y devuelve una matriz (aunque sea unidimensional). Para obtener un vector unidemensional, y operar posteriormente con más agilidad, `similarities_tfidf` ha de almacenar el elemento cero de la matriz devuelta por la función, o transformar esta matriz en un vector. 
</div>

In [ ]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities_tfidf = # COMPLETAR

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Obtener los índices de los 6 libros más similares a `text_query` (obviamente, el más similar será el propio libro).



In [ ]:
top_similarities = # COMPLETAR

df_goodreads.iloc[top_similarities]['title']

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Se aprecia que, efectivamente, los libros son muy similares. De hecho, todos corresponden a la saga. Esto puede deverse al uso de palabras clave muy concretas como _"Twilight"_.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section4"></a> 
## <font color="#00586D"> 4. Recomendación basada en contenido </font>
<br>

Existen muchas posibilidades en la implementación de sistemas de recomendación basados en contenido. En general, todas requieren:

* Determinar qué ítems prefiere el usuario
* Encontrar ítems similares
* Priorizar esos ítems


Para ilustrar el funcionamiento de este tipo de sistemas, se tomarán los datos relativos a la actividad de un usuario escogido al azar, y se devolverá un conjunto de libros recomendados. 


El archivo `'./data/ratings.csv` contiene las valoraciones hechas por más de 53000 usuarios a los 10000 libros. En total, contiene cerca de un millón de entradas en formato (`user_id`, `book_id`, `rating`). El tamaño de la base de datos dificulta el trabajo con una matriz, aunque sea dispersa, ya que el proceso de elaboración (mediante `pivot` es muy lento). 

In [ ]:
df_ratings = pd.read_csv('./data/ratings.csv', sep=',')

display(df_ratings.head())
df_ratings.shape

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>


---

<a id="section41"></a> 
### <font color="#00586D"> Selección de libros preferidos por el usuario </font>
<br>

El primer paso de la recomendación consiste en determinar qué libros prefiere el usuario.  Una posibilidad consiste en seleccionar aquellos para los que éste ha otorgado una puntuación mayor que tres. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Tomar un usuario al azar y devolver las entradas correspondientes a los libros que ha votado. Almacenarlas en el `DataFrame` `ratings_user`. Descartar de `ratings_user` todos los libros con puntuación menor que cuatro. 

In [ ]:
np.random.seed(0)
test_user = np.random.randint(max(df_ratings['user_id']))

ratings_user = # COMPLETAR
ratings_user

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section42"></a> 
### <font color="#00586D"> Búsqueda de libros similares </font>
<br>

Una vez se dispone de la lista de libros, es necesario encontrar libros similares. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font>Extraer la representación de los libros relativos al usuario y copiarla en una matriz denominada `user_books_tfidf`. 

In [ ]:
user_books_tfidf = # COMPLETAR

user_books_tfidf.shape

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Calcular la matriz de similaridades coseno entre `user_books_tfidf`, `data_books_tfidf`. Fijar las similaridades de los libros leídos con el usuario  (`books_user_id `) consigo mismos a -1 con el fin de descartarlos. 

In [ ]:
similarities_user = # COMPLETAR
similarities_user[:,books_user_id]=-1

similarities_user.shape

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener los 3 libros más similares _a cada uno_ de los leídos por el usuario y almacenarlos en la matriz `similar_books`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> La función `similarities_user.argsort(axis=1)` devuelve los índices ordenados (ascendentemente) para cada fila del array. Se trata de coger las 3 últimas columnas de `similarities_user`. 
    
Por otra parte, el resultado de este ejercicio ha de ser un array ***unidimensional*** sin elementos repetidos.
</div>

In [ ]:
similar_books = # COMPLETAR

print(similar_books, "("+str(len(similar_books))+" libros distintos.)")

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section43"></a> 
### <font color="#00586D"> Priorización de resultados y recomendación</font>

Llegados a este punto, se han obtenido los libros más similares a los leídos y valorados positivamente por el usuario. Una posibilidad a la hora de priorizar los resultados consistiría en dar más peso a aquellos libros similares que han aparecido más veces en la matriz `similar_books`. 

También es posible utilizar la información del `DataFrame` `df_goodreads` para priorizar los libros. En concreto, la columna `average_rating` contiene la valoración media de cada libro en la plataforma, y puede ser utilizada para priorizar. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener los 10 libros con mayor `average_rating` de entre los más similares a los valorados positivamente por el usuario (`similar_books`). Almacenar sus índices en un array denominado `rec_books_user_id`.

In [ ]:
rec_books_user_id = # COMPLETAR
rec_books_user_id

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Por último, se imprimen los nombres de los libros preferidos por el usuario, cuyos índices están almacenados en `books_user_id`, y la recomendación, cuyos índices están almacenados en `rec_books_user_id`.

In [ ]:
print("Libros preferidos por el usuario: ")
print("----------------------------------")
for book in df_goodreads.iloc[books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)
    
print("\n\nRecomendación: ")
print("----------------------------------")
for book in df_goodreads.iloc[rec_books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)   

Puede observarse que, efectivamente, los títulos recomendados guardan mucha relación con los títulos valorados positivamente por el usuario. 

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section5"></a> 
##  <font color="#00586D"> 5. Sistema híbrido de recomendación  </font>
<br>

<br> En el apartado anterior se ha utilizado la valoración media de cada libro para priorizar los vecinos más cercanos. Existe otra posibilidad, que consite en utilizar las valoraciones de los libros obtenidas mediante filtrado colaborativo. Esto constituye un sistema híbrido, ya que se mezclan las dos aproximaciones. Por una parte, se seleccionan libros basados en contenido, y de ellos, se muestra el conjunto para el que se predice una mayor puntuación por parte del usuario.
<br>

No existe mucho material disponible en relación a este tipo de modelos. Tampoco una librería de referencia, aunque va cobrando popularidad [surprise](http://surpriselib.com/). En primer lugar, se utilizará esta librería para obtener los scores mediante filtrado colaborativo. En concreto, se utilizará el algoritmo SVD (visto en clase). 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> El autor principal de la librería describe en en una serie de artículos [enlace](http://nicolas-hug.com/blog/) como funciona la versión básica de este algoritmo. La lectura es muy recomendable. 
</div>

En la siguiente celda, se cargan los votos  en la estructura `ratings_data` (usada por `surprise`) y se aprende un modelo SVD.

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection.split import train_test_split

reader = Reader()
ratings_data = Dataset.load_from_df(df_ratings, reader).build_full_trainset()
svd = SVD(n_factors=10)
svd.fit(ratings_data);

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener uno a uno, las valoraciones de los libros similares a los preferidos por el usuario (`similar_books`), y tomar los 10 con más valoración. 

In [ ]:
scores = np.zeros(similar_books.shape)

#
# COMPLETAR
#
    
rec_books_user_id = # COMPLETAR

print("\n\nRecomendación: ")
print("----------------------------------")
for book in df_goodreads.iloc[rec_books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)    

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section6"></a> 
## <font color="#00586D"> 6. LDA y búsqueda por similaridad </font>
<br>

LDA permite modelar los grupos que dan lugar al corpus de documentos, de modo que cada documento se representa como una mixtura de las distribuciones de probabilidad de cada grupo, dada por un vector de pesos. Esta representación puede ser utilizada también para caracterizar el documento (en lugar de _tf-idf_) y permite cuantificar la similaridad con otros. Por tanto, también serviría para implementar un sistema de recomendación (basado en contenido o híbrido) similar al anterior.

LDA trabaja con la matriz de frecuencias de  términos por documento (_term frecuency_). Por tanto, es necesario primero obtenerla. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Construir un objeto `CountVectorizer` de `sklearn` y almacenarlo en la variable `cv_vect`.  Obtener la matriz `data_books_cv` a partir de los datos almacenados en `data_books`. Utilizar un tamaño máximo de vocabulario de 5000 (para que LDA no requiera demasiado tiempo y memoria). Almacenar el vocabulario generado en `cv_vect` en la variable `terms`. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#
# COMPLETAR
#

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

Una vez construida la matriz de frecuencias, se ejecutará el algoritmo LDA, para representar los documentos mediante 20 grupos.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

# Número de temas
n_topics = 20

lda = LatentDirichletAllocation(n_components=n_topics, max_iter=20, learning_method='online',
                                learning_offset=50., random_state=0, n_jobs=-1)

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Construir el modelo LDA y almacenarlo en la variable `topic_term`. Guardar la representación de los documentos en la variable `data_books_lda`.

In [ ]:
from time import time
start = time()

#
# COMPLETAR
#

print("Tiempo: {:0.3f}s.".format(time() - start))
print("Tamaño del modelo:", topic_term.shape)
print("Tamaño del resultado de la transformación: ", data_books_lda.shape)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

La siguiente función (de la práctica de LDA) imprime los términos y scores más importantes para cada grupo.

In [ ]:
def topic_relevant_words(topic_id, model, terms, num_words):
    print("Grupo: ", topic_id)
    print("----------------------")
    # Extrae las posiciones de los mayores scores. 
    top_term_ids = model[topic_id,:].argsort()[:-num_words-1:-1]
    # Extrae los términos de las posiciones correspondientes
    top_terms = [terms[id_term] for id_term in top_term_ids]
    # Extrae y calcula los scores
    top_scores =  model[topic_id,top_term_ids] / np.sum(model[topic_id,:])
    # Imprime los resultados
    for term, score in zip(top_terms,top_scores):
        print("{:s}:     \t{:.4f}".format(term,score))
    print("----------------------\n")

topic_relevant_words(0,topic_term, terms, 10)

#### <font color="#00586D"> Búsqueda del libro más similar </font>

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font>  Obtener la representación a partir del modelo `lda` del documento `text_query`. Almacenarla en la variable `text_query_lda`. 

In [ ]:
text_query_lda = # COMPLETAR
text_query_lda

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Mostrar las características de los tres grupos más relevantes en el texto `text_query`. 

In [ ]:
#
# COMPLETAR
#

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Calcular las similaridades (distancia_euclídea) del documento `text_query_lda` con el resto de documentos en la matriz `data_books_lda`, y almacenarlas en un vector denominado `similarities_lda`.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

similarities_lda = # COMPLETAR

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Obtener los índices de los 10 libros más similares a `text_query`. Hay que tener en cuenta que `euclidean_distances` devuelve distancias, no similaridades, por lo que hay que tomar los primeros índices. 

In [ ]:
top_similarities = # COMPLETAR

df_goodreads.iloc[top_similarities]['title']

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<!--comment -->

En este caso las recomendaciones no son tan evidentes. Sin embargo, se aprecia que la temática es muy parecida a la de _"Twilight"_.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#00586D"></i> </font></div>